In [6]:
import cv2
import numpy as np
import mediapipe as mp
from keras.models import load_model
import time
import csv

In [7]:
def inFrame(lst):
    if lst[28].visibility > 0.7 and lst[27].visibility > 0. and lst[15].visibility > 0.7 and lst[16].visibility > 0.7:
        return True
    return False

model = load_model("model.h5")
label = np.load("labels.npy")

In [8]:
holistic = mp.solutions.pose
holis = holistic.Pose()
drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

In [9]:
# Ask the user to input the desired yoga pose
desired_pose = input("Enter the desired yoga pose for classification: ")

pose_start_time = None
pose_data = []

Enter the desired yoga pose for classification:  tree


In [ ]:
while True:
    lst = []
    _, frm = cap.read()
    frm = cv2.flip(frm, 1)
    frm = cv2.blur(frm, (4, 4))

    res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))

    if res.pose_landmarks and inFrame(res.pose_landmarks.landmark):
        for i in res.pose_landmarks.landmark:
            lst.append(i.x - res.pose_landmarks.landmark[0].x)
            lst.append(i.y - res.pose_landmarks.landmark[0].y)

        lst = np.array(lst).reshape(1, -1)

        p = model.predict(lst)
        pred = label[np.argmax(p)]

        if p[0][np.argmax(p)] >= 0.98 and pred == desired_pose:
            if pose_start_time is None:
                pose_start_time = time.time()
                cv2.putText(frm, f"Correct Pose: {pred}", (180, 180), cv2.FONT_ITALIC, 1.3, (0, 255, 0), 2)
            else:
                pose_duration = time.time() - pose_start_time
                cv2.putText(frm, f"Correct Pose: {pred} | Duration: {pose_duration:.2f} seconds", (180, 180), cv2.FONT_ITALIC, 1.3, (0, 255, 0), 2)
        elif p[0][np.argmax(p)] >= 0.98:
            cv2.putText(frm, f"Incorrect Pose: {pred}", (100, 180), cv2.FONT_ITALIC, 1.8, (0, 0, 255), 3)
            if pose_start_time is not None:
                # Save data to pose_data list
                pose_data.append({"Pose": pred, "Duration": pose_duration, "Date": time.strftime("%Y-%m-%d %H:%M:%S")})
                pose_start_time = None  # Reset the timer if the pose is incorrect
        else:
            cv2.putText(frm, "Asana is either wrong or not trained", (100, 180), cv2.FONT_ITALIC, 1.8, (0, 0, 255), 3)
            pose_start_time = None  # Reset the timer if the pose is not confidently detected
    else:
        cv2.putText(frm, "Make Sure Full body is visible", (100, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
        if pose_start_time is not None:
            # Save data to pose_data list
            pose_data.append({"Pose": "Unknown", "Duration": pose_duration, "Date": time.strftime("%Y-%m-%d %H:%M:%S")})
            pose_start_time = None  # Reset the timer if the full body is not visible

    drawing.draw_landmarks(frm, res.pose_landmarks, holistic.POSE_CONNECTIONS,
                           connection_drawing_spec=drawing.DrawingSpec(color=(255, 255, 255), thickness=6),
                           landmark_drawing_spec=drawing.DrawingSpec(color=(0, 0, 255), circle_radius=3, thickness=3))

    cv2.imshow("Yoga Pose Classification", frm)

    if cv2.waitKey(1) == 27:
        # Save the data to a CSV file
        with open("pose_data.csv", "w", newline="") as csvfile:
            fieldnames = ["Pose", "Duration", "Date"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            # Write the header
            writer.writeheader()

            # Write the data
            for pose_entry in pose_data:
                writer.writerow(pose_entry)

        cv2.destroyAllWindows()
        cap.release()
        break